# JACCO: Jitter-Aware Adaptive Congestion Control Optimization
**Optimized for Satellite Networks (e.g., Starlink)**

In [ ]:
import timeimport numpy as npimport statisticsfrom collections import dequefrom enum import Enumfrom dataclasses import dataclassclass NetworkCondition(Enum):    CLEAN = "clean"    JITTERY = "jittery"    CONGESTED = "congested"    HANDOFF = "handoff"@dataclassclass SatelliteMetrics:    timestamp: float    rtt: float    bandwidth: float    packet_loss: float    handoff_detected: bool = Falseclass JACCOSatelliteController:    def __init__(self, initial_cwnd=10, max_cwnd=65536):        self.cwnd = float(initial_cwnd)        self.ssthresh = max_cwnd // 2        self.max_cwnd = max_cwnd        self.rtt_history = deque(maxlen=25)        self.bw_history = deque(maxlen=15)        self.congestion_indicators = deque(maxlen=8)        self.alpha = 0.7        self.beta = 0.3        self.gamma = 0.125        self.base_rtt = float('inf')        self.estimated_bw = 0.0        self.current_jitter_variance = 0.0        self.detected_condition = NetworkCondition.CLEAN        self.last_handoff_time = 0.0    def update_satellite_measurements(self, metrics: SatelliteMetrics):        current_time = time.time()        if self.rtt_history:            rtt_delta = abs(metrics.rtt - self.rtt_history[-1])            if rtt_delta > 0.020:                self.last_handoff_time = current_time                self.detected_condition = NetworkCondition.HANDOFF                metrics.handoff_detected = True        self.base_rtt = min(self.base_rtt, metrics.rtt)        self.rtt_history.append(metrics.rtt)        if metrics.bandwidth > 0:            self.bw_history.append(metrics.bandwidth)        self._analyze_jitter()        self._update_bandwidth(metrics.bandwidth)        self._classify_network_condition()    def _analyze_jitter(self):        if len(self.rtt_history) < 5:            return        rtts = np.array(list(self.rtt_history))        rtt_diffs = np.diff(rtts)        high_freq_jitter = np.var(rtt_diffs)        if len(rtts) >= 10:            recent_trend = np.mean(rtts[-5:]) - np.mean(rtts[-10:-5])            congestion_indicator = max(0, recent_trend / self.base_rtt)            self.congestion_indicators.append(congestion_indicator)        self.current_jitter_variance = high_freq_jitter    def _update_bandwidth(self, current_bw):        if current_bw <= 0:            return        smoothing = 0.05 if self.detected_condition == NetworkCondition.JITTERY else self.gamma        self.estimated_bw = (1 - smoothing) * self.estimated_bw + smoothing * current_bw    def _classify_network_condition(self):        if time.time() - self.last_handoff_time < 2.0:            self.detected_condition = NetworkCondition.HANDOFF        elif len(self.congestion_indicators) >= 3 and np.mean(self.congestion_indicators[-3:]) > 0.1:            self.detected_condition = NetworkCondition.CONGESTED        elif self.current_jitter_variance > 0.01:            self.detected_condition = NetworkCondition.JITTERY        else:            self.detected_condition = NetworkCondition.CLEAN    def get_cwnd_update(self, ack=True, loss=False):        if loss:            if self.detected_condition == NetworkCondition.HANDOFF:                self.cwnd *= 0.9            else:                self.ssthresh = max(self.cwnd // 2, 2)                self.cwnd = self.ssthresh            return self.cwnd        congestion_factor = 1.0 / (1.0 + self.alpha * max(0, (self.rtt_history[-1] - self.base_rtt) / self.base_rtt))        jitter_penalty = self.beta * (self.current_jitter_variance / self.base_rtt)        net_factor = congestion_factor - jitter_penalty        net_factor = max(0.05, min(2.0, net_factor))        if self.cwnd < self.ssthresh:            self.cwnd += 1.2 * net_factor        else:            self.cwnd += max(0.1, net_factor) / self.cwnd        self.cwnd = min(self.max_cwnd, max(1.0, self.cwnd))        return self.cwnd# Simulation Exampleif __name__ == "__main__":    controller = JACCOSatelliteController()    for t in range(60):        simulated_rtt = 0.045 + 0.010 * np.sin(2 * np.pi * t / 60) + np.random.normal(0, 0.003)        simulated_bw = 100_000_000        metrics = SatelliteMetrics(time.time(), simulated_rtt, simulated_bw, 0.0)        controller.update_satellite_measurements(metrics)        cwnd = controller.get_cwnd_update()        print(f"t={t:02d}s | CWND={cwnd:.2f} | Condition={controller.detected_condition.value}")